In [1]:
from transformers import BertTokenizer, T5ForConditionalGeneration, Text2TextGenerationPipeline


加载模型和tokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained("../uer/t5-base-chinese-cluecorpussmall")
model = T5ForConditionalGeneration.from_pretrained("../uer/t5-base-chinese-cluecorpussmall")

d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\modeling_utils.py:442: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_lo

加载数据集

In [3]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="../data/DuReaderQG/train.json", split="train")
datasets = dataset.train_test_split(test_size=0.2)
dev_dataset = load_dataset("json", data_files="../data/DuReaderQG/dev.json", split="train")
datasets, dev_dataset

(DatasetDict({
     train: Dataset({
         features: ['context', 'answer', 'question', 'id'],
         num_rows: 11616
     })
     test: Dataset({
         features: ['context', 'answer', 'question', 'id'],
         num_rows: 2904
     })
 }),
 Dataset({
     features: ['context', 'answer', 'question', 'id'],
     num_rows: 984
 }))

In [4]:
tokenize_example = tokenizer("context: " + datasets["train"][0]["context"] + "question: " + datasets["train"][0]["question"], return_tensors="pt", max_length=256, truncation=True, padding="max_length")
tokenize_example

{'input_ids': tensor([[  101, 11485, 11816,   131,   679,  5314,   872,  6432,  4415,  6389,
           966,   749,   117,   671,  2828,  2398,  2382,   886,  4500,   117,
          6438,  1091,  8183,   118,   100,   120,   100,   117,  3152,   816,
          6632,  2207,  6632,  1914,   117,  6862,  2428,  6632,  2714,   100,
          4638,  6413,  2218,  3221,   679,  1168,   122,  4907,   170,  6821,
           702,  6656,   792,  6574,  2166,  2428,  3300,  1068,  5143,   117,
           117,  6760,  6862,  3221,   671,   702,  2900,  3403,   117,  2166,
          2428,  3221,  1369,   671,   702,  2900,  3403,   117,  6438,  1091,
          6862,  2428,  2218,  3221,  1296,   855,  3198,  7313,  4828,  1928,
          2812,  6814,  4638,  7481,  4916,   733,   809,  3144,  2945,  2166,
          2428,   117,  4385,  1762,  1296,  4817,   122,  8165,  4638,  3683,
          3193,  3309,  1296,  4817,   100,  4638,  2571,   679,  2208,   511,
           671,  5663,  2769,   812,  

In [5]:
label = tokenizer(datasets["train"][0]["answer"], max_length=64, truncation=True, padding="max_length")
label["input_ids"][label["input_ids"] == tokenizer.pad_token_id] = -100
label

{'input_ids': [-100, 8183, 118, 100, 120, 100, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [6]:
tokenize_example["labels"] = label["input_ids"]
tokenize_example

{'input_ids': tensor([[  101, 11485, 11816,   131,   679,  5314,   872,  6432,  4415,  6389,
           966,   749,   117,   671,  2828,  2398,  2382,   886,  4500,   117,
          6438,  1091,  8183,   118,   100,   120,   100,   117,  3152,   816,
          6632,  2207,  6632,  1914,   117,  6862,  2428,  6632,  2714,   100,
          4638,  6413,  2218,  3221,   679,  1168,   122,  4907,   170,  6821,
           702,  6656,   792,  6574,  2166,  2428,  3300,  1068,  5143,   117,
           117,  6760,  6862,  3221,   671,   702,  2900,  3403,   117,  2166,
          2428,  3221,  1369,   671,   702,  2900,  3403,   117,  6438,  1091,
          6862,  2428,  2218,  3221,  1296,   855,  3198,  7313,  4828,  1928,
          2812,  6814,  4638,  7481,  4916,   733,   809,  3144,  2945,  2166,
          2428,   117,  4385,  1762,  1296,  4817,   122,  8165,  4638,  3683,
          3193,  3309,  1296,  4817,   100,  4638,  2571,   679,  2208,   511,
           671,  5663,  2769,   812,  

数据预处理

In [26]:
def process_function(examples):
    context_questions = [f"context:{context} question:{question}" for context, question in zip(examples["context"], examples["question"])]
    tokenized_examples = tokenizer(context_questions, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["answer"], max_length=64, truncation=True, padding="max_length")
    labels["input_ids"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in labels["input_ids"]
    ]
    tokenized_examples["labels"] = labels["input_ids"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11616
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2904
    })
})

In [8]:
tokenized_datasets["train"][0]

{'input_ids': [101,
  11485,
  11816,
  131,
  679,
  5314,
  872,
  6432,
  4415,
  6389,
  966,
  749,
  117,
  671,
  2828,
  2398,
  2382,
  886,
  4500,
  117,
  6438,
  1091,
  8183,
  118,
  100,
  120,
  100,
  117,
  3152,
  816,
  6632,
  2207,
  6632,
  1914,
  117,
  6862,
  2428,
  6632,
  2714,
  100,
  4638,
  6413,
  2218,
  3221,
  679,
  1168,
  122,
  4907,
  170,
  6821,
  702,
  6656,
  792,
  6574,
  2166,
  2428,
  3300,
  1068,
  5143,
  117,
  117,
  6760,
  6862,
  3221,
  671,
  702,
  2900,
  3403,
  117,
  2166,
  2428,
  3221,
  1369,
  671,
  702,
  2900,
  3403,
  117,
  6438,
  1091,
  6862,
  2428,
  2218,
  3221,
  1296,
  855,
  3198,
  7313,
  4828,
  1928,
  2812,
  6814,
  4638,
  7481,
  4916,
  733,
  809,
  3144,
  2945,
  2166,
  2428,
  117,
  4385,
  1762,
  1296,
  4817,
  122,
  8165,
  4638,
  3683,
  3193,
  3309,
  1296,
  4817,
  100,
  4638,
  2571,
  679,
  2208,
  511,
  671,
  5663,
  2769,
  812,
  5543,
  2697,
  6230,
  4638,
  

In [27]:
from transformers import TrainingArguments

train_args = TrainingArguments(
    output_dir="./output/generateAskAnswer",
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
)

In [28]:
from transformers import Trainer, DefaultDataCollator

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DefaultDataCollator(),
    tokenizer=tokenizer,
)

In [29]:
trainer.train()

d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/7260 [00:00<?, ?it/s]

{'loss': 9.1569, 'learning_rate': 9.986225895316806e-05, 'epoch': 0.03}
{'loss': 8.7968, 'learning_rate': 9.972451790633609e-05, 'epoch': 0.06}
{'loss': 8.4562, 'learning_rate': 9.958677685950414e-05, 'epoch': 0.08}
{'loss': 8.2263, 'learning_rate': 9.944903581267218e-05, 'epoch': 0.11}
{'loss': 8.0821, 'learning_rate': 9.931129476584022e-05, 'epoch': 0.14}
{'loss': 7.9258, 'learning_rate': 9.917355371900827e-05, 'epoch': 0.17}
{'loss': 7.7649, 'learning_rate': 9.903581267217631e-05, 'epoch': 0.19}
{'loss': 7.6729, 'learning_rate': 9.889807162534436e-05, 'epoch': 0.22}
{'loss': 7.5566, 'learning_rate': 9.876033057851241e-05, 'epoch': 0.25}
{'loss': 7.4418, 'learning_rate': 9.862258953168044e-05, 'epoch': 0.28}
{'loss': 7.3313, 'learning_rate': 9.848484848484849e-05, 'epoch': 0.3}
{'loss': 7.2631, 'learning_rate': 9.834710743801654e-05, 'epoch': 0.33}
{'loss': 7.1633, 'learning_rate': 9.820936639118457e-05, 'epoch': 0.36}
{'loss': 7.0557, 'learning_rate': 9.807162534435263e-05, 'epoch':

  0%|          | 0/363 [00:00<?, ?it/s]

{'eval_loss': 5.653988838195801, 'eval_runtime': 96.0189, 'eval_samples_per_second': 30.244, 'eval_steps_per_second': 3.781, 'epoch': 1.0}
{'loss': 6.0263, 'learning_rate': 9.490358126721764e-05, 'epoch': 1.02}
{'loss': 5.9873, 'learning_rate': 9.476584022038568e-05, 'epoch': 1.05}
{'loss': 6.0252, 'learning_rate': 9.462809917355372e-05, 'epoch': 1.07}
{'loss': 5.9483, 'learning_rate': 9.449035812672177e-05, 'epoch': 1.1}
{'loss': 5.9569, 'learning_rate': 9.435261707988981e-05, 'epoch': 1.13}
{'loss': 5.9447, 'learning_rate': 9.421487603305785e-05, 'epoch': 1.16}
{'loss': 5.8284, 'learning_rate': 9.40771349862259e-05, 'epoch': 1.18}
{'loss': 5.8181, 'learning_rate': 9.393939393939395e-05, 'epoch': 1.21}
{'loss': 5.8048, 'learning_rate': 9.380165289256199e-05, 'epoch': 1.24}
{'loss': 5.8205, 'learning_rate': 9.366391184573004e-05, 'epoch': 1.27}
{'loss': 5.7426, 'learning_rate': 9.352617079889807e-05, 'epoch': 1.29}
{'loss': 5.74, 'learning_rate': 9.338842975206612e-05, 'epoch': 1.32}
{

...

  0%|          | 0/363 [00:00<?, ?it/s]

{'eval_loss': 1.6686396598815918, 'eval_runtime': 96.8244, 'eval_samples_per_second': 29.992, 'eval_steps_per_second': 3.749, 'epoch': 10.0}
{'loss': 2.0018, 'learning_rate': 4.986225895316804e-05, 'epoch': 10.03}
{'loss': 1.9277, 'learning_rate': 4.972451790633609e-05, 'epoch': 10.06}
{'loss': 2.0104, 'learning_rate': 4.958677685950414e-05, 'epoch': 10.08}
{'loss': 1.969, 'learning_rate': 4.944903581267218e-05, 'epoch': 10.11}
{'loss': 2.0687, 'learning_rate': 4.931129476584022e-05, 'epoch': 10.14}
{'loss': 1.9572, 'learning_rate': 4.917355371900827e-05, 'epoch': 10.17}
{'loss': 1.9963, 'learning_rate': 4.903581267217631e-05, 'epoch': 10.19}
{'loss': 1.9722, 'learning_rate': 4.889807162534435e-05, 'epoch': 10.22}
{'loss': 2.0101, 'learning_rate': 4.87603305785124e-05, 'epoch': 10.25}
{'loss': 1.9424, 'learning_rate': 4.862258953168045e-05, 'epoch': 10.28}
{'loss': 1.9768, 'learning_rate': 4.848484848484849e-05, 'epoch': 10.3}
{'loss': 1.9815, 'learning_rate': 4.834710743801653e-05, 'e

  0%|          | 0/363 [00:00<?, ?it/s]

{'eval_loss': 1.5750449895858765, 'eval_runtime': 92.0151, 'eval_samples_per_second': 31.56, 'eval_steps_per_second': 3.945, 'epoch': 11.0}
{'loss': 1.9775, 'learning_rate': 4.4903581267217634e-05, 'epoch': 11.02}


KeyboardInterrupt: 

In [13]:
# text2text_generator = Text2TextGenerationPipeline(model, tokenizer)  
# text2text_generator("中国的首都是extra0京", max_length=50, do_sample=False)

In [24]:
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

train_loader = DataLoader(tokenized_datasets["train"], batch_size=8, shuffle=True)
batch = next(iter(train_loader))
print(batch.keys())
print('batch shape:', {k: v.shape for k, v in batch.items()})
print(batch)
test_loader = DataLoader(tokenized_datasets["test"], batch_size=8, shuffle=False)
# 定义损失函数和优化器
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# 定义BLEU评分函数
def calculate_bleu(reference, candidate):
    reference = [reference]
    candidate = candidate
    smoothie = SmoothingFunction().method4
    bleu1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    bleu2 = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
    bleu3 = sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
    bleu4 = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    return bleu1, bleu2, bleu3, bleu4

# 训练和验证
num_epochs = 20
train_losses = []
test_bleu1 = []
test_bleu2 = []
test_bleu3 = []
test_bleu4 = []

model.train()
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch, batch_data in enumerate(train_loader):
        print(f"Batch {batch}: {type(batch_data)}, {batch_data.keys() if isinstance(batch_data, dict) else batch_data}")
        optimizer.zero_grad()
        batch_data = batch_data.to(model.device)
        ouputs = model(**batch_data)
        # input_ids = batch_data["input_ids"].to(model.device)
        # attention_mask = batch_data["attention_mask"].to(model.device)
        # labels = batch_data["labels"].to(model.device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    # 验证
    model.eval()
    total_bleu1, total_bleu2, total_bleu3, total_bleu4 = 0, 0, 0, 0
    with torch.no_grad():
        for batch, batch_data in enumerate(test_loader):
            input_ids = batch_data["input_ids"].to(model.device)
            attention_mask = batch_data["attention_mask"].to(model.device)
            labels = batch_data["labels"].to(model.device)
            
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=64)
            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            
            for pred, label in zip(decoded_preds, decoded_labels):
                bleu1, bleu2, bleu3, bleu4 = calculate_bleu(label.split(), pred.split())
                total_bleu1 += bleu1
                total_bleu2 += bleu2
                total_bleu3 += bleu3
                total_bleu4 += bleu4
    
    avg_bleu1 = total_bleu1 / len(test_loader.dataset)
    avg_bleu2 = total_bleu2 / len(test_loader.dataset)
    avg_bleu3 = total_bleu3 / len(test_loader.dataset)
    avg_bleu4 = total_bleu4 / len(test_loader.dataset)
    
    test_bleu1.append(avg_bleu1)
    test_bleu2.append(avg_bleu2)
    test_bleu3.append(avg_bleu3)
    test_bleu4.append(avg_bleu4)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, BLEU-1: {avg_bleu1:.4f}, BLEU-2: {avg_bleu2:.4f}, BLEU-3: {avg_bleu3:.4f}, BLEU-4: {avg_bleu4:.4f}")

# 绘制收敛曲线图
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(test_bleu1, label='BLEU-1')
plt.plot(test_bleu2, label='BLEU-2')
plt.plot(test_bleu3, label='BLEU-3')
plt.plot(test_bleu4, label='BLEU-4')
plt.xlabel('Epoch')
plt.ylabel('BLEU Score')
plt.legend()

plt.tight_layout()
plt.show()

# 预测代码
def generate_answer(context, question):
    input_text = f"context:{context} question:{question}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length").input_ids.to(model.device)
    output_ids = model.generate(input_ids, max_length=64, do_sample=False)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# 示例预测
context = dev_dataset["context"][0]
question = dev_dataset["question"][0]
print(generate_answer(context, question))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


AttributeError: 'list' object has no attribute 'shape'